This is an example from simpy.

In [10]:
import random
import numpy as np
import simpy

In [ ]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print(f'{arrive:7.4f} {name}: Here I am')

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print(f'{env.now:7.4f} {name}: Waited {wait:6.3f}')

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'{env.now:7.4f} {name}: Finished')

        else:
            # We reneged
            print(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


The following is from ChatGPT.

In [13]:
def exponential(x):  # function of exponential distribution 
    return random.expovariate(x)

def deterministic(x):   # function of deterministic time
    return x 

def hyperexponential(A):   # fucntion takes tuple
    a , b = A[0], A[1]
    z=random.random()     # create random number from (0,1)
    if z < 0.75:         # with 75% chance, generate exponential distribution number with mean 1/a 
        x = random.expovariate(a)
    else:
        x = random.expovariate(b)    # with 25% chance, generate exponential distribution number with mean 1/b

    return x

In [3]:
import simpy
import random

# Customer class to model the customers arriving at the system
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.wait_time = 0
        self.service_time = 0
        self.system_time = 0

def customer_arrivals(env, servers, sim_duration, function_arrival_times, function_service_times):
    """Handles the arrival of customers."""
    customer_id = 0
    while env.now < sim_duration:
        customer_id += 1
        arrival_time = env.now
        customer = Customer(arrival_time)
        print(f"Time {env.now:.2f}: Customer {customer_id} arrives.")
        env.process(customer_service(env, customer, servers, customer_id, function_service_times))
        yield env.timeout(function_arrival_times(l))  # Exponential inter-arrival time

def customer_service(env, customer, servers, customer_id, function_service_times):
    """Simulates the service of a customer."""
    # Request a server
    with servers.request() as request:
        print(f"Time {env.now:.2f}: Customer {customer_id} waits for a server.")
        yield request  # Wait for a free server
        
        # Customer starts service
        wait_time = env.now - customer.arrival_time
        customer.wait_time = wait_time
        service_time = function_service_times(u)  # Exponential service time
        customer.service_time = service_time
        print(f"Time {env.now:.2f}: Customer {customer_id} starts service. Wait time = {wait_time:.2f}, Service time = {service_time:.2f}")

        # Simulate service time
        yield env.timeout(service_time)
        
        # Customer completes service
        customer.system_time = env.now - customer.arrival_time
        print(f"Time {env.now:.2f}: Customer {customer_id} finishes service. Total time in system = {customer.system_time:.2f}")

        # Store results
        waiting_times.append(customer.wait_time)
        system_times.append(customer.system_time)


Lets run simulation with exponential arrival and service times.

In [4]:
# Parameters
l = 4  # Rate of customer arrivals (lambda)
u = 1  # Rate of service (mu)
m = 5  # Number of servers
sim_duration = 10  # Simulation duration (in time units)

# Lists to store results
waiting_times = []
system_times = []


# Initialize the environment and server pool
env = simpy.Environment()
servers = simpy.Resource(env, capacity=m)

# Start the arrival process
env.process(customer_arrivals(env, servers, sim_duration, exponential, exponential))

# Run the simulation
env.run()

# Output the results
print(f"\nAverage waiting time: {sum(waiting_times) / len(waiting_times):.2f}")
print(f"Average system time: {sum(system_times) / len(system_times):.2f}")
print(f"Total customers served: {len(waiting_times)}")

Time 0.00: Customer 1 arrives.
Time 0.00: Customer 1 waits for a server.
Time 0.00: Customer 1 starts service. Wait time = 0.00, Service time = 0.79
Time 0.26: Customer 2 arrives.
Time 0.26: Customer 2 waits for a server.
Time 0.26: Customer 2 starts service. Wait time = 0.00, Service time = 0.89
Time 0.32: Customer 3 arrives.
Time 0.32: Customer 3 waits for a server.
Time 0.32: Customer 3 starts service. Wait time = 0.00, Service time = 0.01
Time 0.33: Customer 3 finishes service. Total time in system = 0.01
Time 0.74: Customer 4 arrives.
Time 0.74: Customer 4 waits for a server.
Time 0.74: Customer 4 starts service. Wait time = 0.00, Service time = 1.20
Time 0.79: Customer 1 finishes service. Total time in system = 0.79
Time 1.15: Customer 5 arrives.
Time 1.15: Customer 5 waits for a server.
Time 1.15: Customer 5 starts service. Wait time = 0.00, Service time = 0.17
Time 1.15: Customer 2 finishes service. Total time in system = 0.89
Time 1.25: Customer 6 arrives.
Time 1.25: Customer 

Let's run simulation with deterministic times.

In [16]:
# Parameters
l = 1  # time between arrivals
u = 3  # time of service
m = 2  # Number of servers
sim_duration = 10  # Simulation duration (in time units)

# Lists to store results
waiting_times = []
system_times = []


# Initialize the environment and server pool
env = simpy.Environment()
servers = simpy.Resource(env, capacity=m)

# Start the arrival process
env.process(customer_arrivals(env, servers, sim_duration, exponential, deterministic))

# Run the simulation
env.run()

# Output the results
print(f"\nAverage waiting time: {sum(waiting_times) / len(waiting_times):.2f}")
print(f"Average system time: {sum(system_times) / len(system_times):.2f}")
print(f"Total customers served: {len(waiting_times)}")

Time 0.00: Customer 1 arrives.
Time 0.00: Customer 1 waits for a server.
Time 0.00: Customer 1 starts service. Wait time = 0.00, Service time = 3.00
Time 2.13: Customer 2 arrives.
Time 2.13: Customer 2 waits for a server.
Time 2.13: Customer 2 starts service. Wait time = 0.00, Service time = 3.00
Time 3.00: Customer 1 finishes service. Total time in system = 3.00
Time 4.24: Customer 3 arrives.
Time 4.24: Customer 3 waits for a server.
Time 4.24: Customer 3 starts service. Wait time = 0.00, Service time = 3.00
Time 4.70: Customer 4 arrives.
Time 4.70: Customer 4 waits for a server.
Time 4.87: Customer 5 arrives.
Time 4.87: Customer 5 waits for a server.
Time 5.13: Customer 2 finishes service. Total time in system = 3.00
Time 5.13: Customer 4 starts service. Wait time = 0.42, Service time = 3.00
Time 6.67: Customer 6 arrives.
Time 6.67: Customer 6 waits for a server.
Time 7.24: Customer 3 finishes service. Total time in system = 3.00
Time 7.24: Customer 5 starts service. Wait time = 2.37

Let's run a simulation with hyperexponential times.

In [15]:
# Parameters
l = 4  # time between arrivals
u = np.array([1, 5])  # time of service
m = 1  # Number of servers
sim_duration = 10  # Simulation duration (in time units)

# Lists to store results
waiting_times = []
system_times = []


# Initialize the environment and server pool
env = simpy.Environment()
servers = simpy.Resource(env, capacity=m)

# Start the arrival process
env.process(customer_arrivals(env, servers, sim_duration, exponential, hyperexponential))

# Run the simulation
env.run()

# Output the results
print(f"\nAverage waiting time: {sum(waiting_times) / len(waiting_times):.2f}")
print(f"Average system time: {sum(system_times) / len(system_times):.2f}")
print(f"Total customers served: {len(waiting_times)}")

Time 0.00: Customer 1 arrives.
Time 0.00: Customer 1 waits for a server.
Time 0.00: Customer 1 starts service. Wait time = 0.00, Service time = 0.94
Time 0.09: Customer 2 arrives.
Time 0.09: Customer 2 waits for a server.
Time 0.13: Customer 3 arrives.
Time 0.13: Customer 3 waits for a server.
Time 0.49: Customer 4 arrives.
Time 0.49: Customer 4 waits for a server.
Time 0.68: Customer 5 arrives.
Time 0.68: Customer 5 waits for a server.
Time 0.94: Customer 1 finishes service. Total time in system = 0.94
Time 0.94: Customer 2 starts service. Wait time = 0.85, Service time = 0.00
Time 0.94: Customer 2 finishes service. Total time in system = 0.85
Time 0.94: Customer 3 starts service. Wait time = 0.81, Service time = 0.02
Time 0.96: Customer 3 finishes service. Total time in system = 0.83
Time 0.96: Customer 4 starts service. Wait time = 0.47, Service time = 0.42
Time 1.06: Customer 6 arrives.
Time 1.06: Customer 6 waits for a server.
Time 1.38: Customer 4 finishes service. Total time in 

Let's simulate a shortest job first queue.

In [65]:
class customer_priority:
    def __init__(self, env, l, u, m, sim_duration):
        self.env = env
        self.sim_duration = sim_duration
        self.l = l
        self.u = u
        self.m = m
        self.servers = simpy.Resource(env, capacity=self.m)
        self.queue = []
        self.waiting_times = []
    
    def arrival(self):
        while self.env.now < sim_duration:
            yield self.env.timeout(exponential(l))

            arrival_time = self.env.now
            service_time = exponential(u)
            self.queue.append((arrival_time, service_time))
            self.queue.sort(key = lambda x: x[1])

            self.env.process(customer_priority.service(self))

    def service(self):
        while self.queue:
            arrival_time, service_time = self.queue[0]
            self.queue.pop(0)
            waiting_time = self.env.now - arrival_time
            self.waiting_times.append(waiting_time)
            yield self.env.timeout(service_time)
    
    def average_waiting(self):
        self.env.process(customer_priority.arrival(self))
        self.env.run()
        n = len(waiting_times)
        mean_waiting_times = sum(self.waiting_times)/n
        desv_waiting_times = np.sqrt(sum((self.waiting_times-mean_waiting_times)**2)/(n-1))

        return mean_waiting_times, desv_waiting_times

In [66]:
# Parameters
l = 3
u = 1
m = 4
sim_duration = 10

env = simpy.Environment()
results = customer_priority(env, l, u, m, sim_duration)
mean, desv = results.average_waiting()
print(f"\nAverage waiting time: {mean:.2f}")
print(f"\nStandard deviation waiting time: {desv:.2f}")

ZeroDivisionError: float division by zero

Lets try again with not OOP.

In [ ]:
# Customer class to model the customers arriving at the system
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.wait_time = 0
        self.service_time = 0
        self.system_time = 0
        self.queue = []

def customer_arrivals(env, servers, sim_duration, function_arrival_times, function_service_times):
    """Handles the arrival of customers."""
    customer_id = 0
    while env.now < sim_duration:
        customer_id += 1
        arrival_time = env.now
        service_time = function_service_times(u)
        self.queue.append((customer_id, service_time))
        customer = Customer(arrival_time)
        print(f"Time {env.now:.2f}: Customer {customer_id} arrives.")
        env.process(customer_service(env, customer, servers, customer_id, function_service_times))
        yield env.timeout(function_arrival_times(l))  # Exponential inter-arrival time

def customer_service(env, customer, servers, customer_id, function_service_times):
    """Simulates the service of a customer."""
    # Request a server
    while self.queue:
        print(f"Time {env.now:.2f}: Customer {customer_id} waits for a server.")
        yield request  # Wait for a free server
        
        # Customer starts service
        wait_time = env.now - customer.arrival_time
        customer.wait_time = wait_time
        customer.service_time = service_time

        print(f"Time {env.now:.2f}: Customer {customer_id} starts service. Wait time = {wait_time:.2f}, Service time = {service_time:.2f}")

        # Simulate service time
        yield env.timeout(service_time)
        
        # Customer completes service
        customer.system_time = env.now - customer.arrival_time
        print(f"Time {env.now:.2f}: Customer {customer_id} finishes service. Total time in system = {customer.system_time:.2f}")

        # Store results
        waiting_times.append(customer.wait_time)
        system_times.append(customer.system_time)

Zaina's code:

In [67]:
class MM1SJFQueueSimulation:
    def __init__(self, env, lambd, mu):
        self.env = env
        self.lambd = lambd
        self.mu = mu
        self.waiting_times = []
        self.queue = []
        self.server_busy = False

    def arrival_process(self):
        while True:
            # Calculate time until next arrival
            yield self.env.timeout(random.expovariate(self.lambd))
            
            # Calculate service and arrival times to sort shortest job
            arrival_time = self.env.now
            service_time = random.expovariate(self.mu)
            
            # Stores and sorts jobs based on service time into queue 
            self.queue.append((arrival_time, service_time))
            self.queue.sort(key=lambda x: x[1])
            
            # If server is free, continue to the service process
            if not self.server_busy:
                self.env.process(self.service_process())

    def service_process(self):
        while self.queue:
            # Completes service process and removes shortest job from queue when done
            self.server_busy = True
            arrival_time, service_time = self.queue.pop(0)
            waiting_time = self.env.now - arrival_time
            self.waiting_times.append(waiting_time)
            
            yield self.env.timeout(service_time)
        
        self.server_busy = False

    def average_wait_time_run(self, simulation_time):
        self.env.process(self.arrival_process())
        self.env.run(until=simulation_time)
        avg_waiting_time = np.mean(self.waiting_times) if self.waiting_times else 0
        
        return avg_waiting_time

In [ ]:
lambd = 1.5 
mu = 0.6   
simulation_time = 100

env = simpy.Environment()
simulation2 = MM1SJFQueueSimulation(env, lambd, mu)
results2 = simulation2.average_wait_time_run(simulation_time)

print(f'Average Waiting Time for Shortest Job First Simulation (n=1): {results2: .2f}')

Average Waiting Time for Shortest Job First Simulation (n=1):  23.69


This is a try of making object oriented program.

In [42]:
# Customer class to model the customers arriving at the system
class Customer_SJF:
    def __init__(self, env, function_arrival_times, l, function_service_times, u, m, sim_duration):
        self.env = env
        self.sim_duration = sim_duration
        self.l = l
        self.u = u
        self.m = m
        self.servers = simpy.Resource(env, capacity=self.m)
        self.wait_time = 0
        self.service_time = 0
        self.system_time = 0
        self.events = []

    def customer_arrivals(self):
        """Handles the arrival of customers."""
        customer_id = 0
        while self.env.now < sim_duration:
            customer_id += 1
            self.arrival_time = self.env.now
            self.service_time = function_service_times(u)  
            self.events.append((self.arrival_time, self.service_time))
            self.events.sort(key= lambda x: x[1])

            #customer = Customer(self.arrival_time)
            print(f"Time {env.now:.2f}: Customer {customer_id} arrives.")

            self.env.process(costumer_service())
            yield self.env.timeout(function_arrival_times(l))  # Exponential inter-arrival time

    def customer_service(self):
        """Simulates the service of a customer."""
        # Request a server
        with servers.request() as request:
            print(f"Time {env.now:.2f}: Customer {customer_id} waits for a server.")
            yield request  # Wait for a free server
        
        # Customer starts service
            self.wait_time = self.env.now - customer.arrival_time
            customer.wait_time = self.wait_time
            self.service_time = function_service_times(u)  # Exponential service time
            customer.service_time = self.service_time
            print(f"Time {env.now:.2f}: Customer {customer_id} starts service. Wait time = {wait_time:.2f}, Service time = {service_time:.2f}")

        # Simulate service time
            yield env.timeout(self.service_time)
        
        # Customer completes service
            customer.system_time = self.env.now - customer.arrival_time
            print(f"Time {env.now:.2f}: Customer {customer_id} finishes service. Total time in system = {customer.system_time:.2f}")

        # Store results
            waiting_times.append(customer.wait_time)
            system_times.append(customer.system_time)

In [47]:
# Parameters
l = 4  # Rate of customer arrivals (lambda)
u = 1  # Rate of service (mu)
m = 5  # Number of servers
sim_duration = 10  # Simulation duration (in time units)

# Initialize the environment and server pool
env = simpy.Environment()

# Start the arrival process
env.process(Customer_SJF( env, exponential, l, exponential, u, m, sim_duration))

env.run()

ValueError: <__main__.Customer_SJF object at 0x7ac6bccaab10> is not a generator.